In [143]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


In [144]:
dir_name = "../ics_cwe/"
models = ["Gpt2", "pretrained_Gpt2", "SecBert", "pretrained_SecBert", "SecureBert", "pretrained_SecureBert"]
text_embeddings = []
our_embeddings = []
text_matrix = []
our_emb_matrix = []
for model in models:
    text_embeddings.append(np.load(dir_name+model+"/data/all_embeddings.npy"))
    our_embeddings.append(np.load(dir_name+model+"/data/our_embeddings.npy"))
    text_matrix.append(cosine_similarity(text_embeddings[-1]))
    our_emb_matrix.append(cosine_similarity(our_embeddings[-1]))

In [145]:
text_matrix[0].shape

(1136, 1136)

In [146]:
our_emb_matrix[0].shape

(1136, 1136)

In [147]:
weakness_hop_sim_matrix = np.load('../ics_cwe/weakness_hop_sim_matrix.npy')
attack_hop_sim_matrix = np.load('../ics_cwe/attack_hop_sim_matrix.npy')

In [148]:
import numpy as np
import matplotlib.pyplot as plt

def top_cos_sim(matrix1,matrix2, title):

    # Initialize a dictionary to store average hit@k for each k
    

    # models = ["Gpt2", "pretrained_Gpt2", "SecBert", "pretrained_SecBert", "SecureBert", "pretrained_SecureBert"]
    models = ["Gpt2"]
    for j,model in enumerate(models):
        # Define the cosine similarity matrices
        cosine_matrix1 = matrix1[j]
        cosine_matrix2 = matrix2[j]
        average_hit_at_k = {}
        # Loop over each k from 10 to 1136
        
        for k in range(10, 500,50):
            hit_at_k = []
        
            # Loop over each row
            # for i in range(1136):
            mx = [0,0]
            mx_text = [0,0]
            mx_our = [0,0]
            for i in range(203):
                # Get the indices of the top k values in the row for both matrices
                top_k_indices_1 = np.argsort(cosine_matrix1[i])[-k:]
                top_k_indices_2 = np.argsort(cosine_matrix2[i])[-k:]
                only_weak1 = top_k_indices_1[top_k_indices_1>=203]
                only_weak2 = top_k_indices_2[top_k_indices_2>=203]
                if(len(only_weak1)>mx_text[0]):
                    mx_text[0]=len(only_weak1)
                    mx_text[1]=i
                if(len(only_weak2)>mx_our[0]):
                    mx_our[0]=len(only_weak2)
                    mx_our[1]=i
                # Calculate the hit@k (number of common indices in top k)
                hit_count = len(set(only_weak1) & set(only_weak2))
                if(hit_count>mx[0]):
                    mx[0]=hit_count
                    mx[1]=i
            print("k: ",k," top hit ",mx, " max text: ", mx_text, " max our: ",mx_our)

            



In [149]:
top_cos_sim(text_matrix,our_emb_matrix, "")

k:  10  top hit  [4, 136]  max text:  [8, 138]  max our:  [9, 62]
k:  60  top hit  [28, 62]  max text:  [50, 180]  max our:  [57, 110]
k:  110  top hit  [52, 62]  max text:  [98, 180]  max our:  [106, 171]
k:  160  top hit  [84, 138]  max text:  [143, 180]  max our:  [152, 129]
k:  210  top hit  [114, 138]  max text:  [190, 180]  max our:  [197, 129]
k:  260  top hit  [144, 138]  max text:  [236, 180]  max our:  [242, 129]
k:  310  top hit  [172, 138]  max text:  [280, 180]  max our:  [286, 119]
k:  360  top hit  [214, 45]  max text:  [327, 180]  max our:  [333, 193]
k:  410  top hit  [243, 138]  max text:  [372, 157]  max our:  [378, 180]
k:  460  top hit  [283, 138]  max text:  [416, 157]  max our:  [423, 180]


In [161]:
np.argsort(text_matrix[0][180])[-20:]

array([ 271,  261,  967, 1089, 1088,  394,  395,  301,  968,  272,  248,
        784,  532,  247,   98,   54,  149,  201,  157,  180])

In [162]:
np.sort(text_matrix[0][180])[-20:]

array([0.4212988 , 0.42268434, 0.42370372, 0.42494899, 0.42870585,
       0.43041853, 0.43290311, 0.43934435, 0.44070803, 0.44710088,
       0.45081399, 0.46298037, 0.4683687 , 0.48478989, 0.50231863,
       0.5135854 , 0.51901524, 0.53019314, 0.64710249, 1.        ])

In [164]:
np.argsort(our_emb_matrix[0][180])[-20:]

array([1093,  600,  274,  395, 1111,  850, 1054,  261,  272,  808,  201,
        149, 1075, 1094,  291,  372,  968,  370,  157,  180])

In [165]:
set(np.argsort(our_emb_matrix[0][180])[-20:])&set(np.argsort(text_matrix[0][180])[-20:])

{149, 157, 180, 201, 261, 272, 395, 968}

In [58]:
np.sort(our_emb_matrix[0][138])[-10:]

array([0.7037285 , 0.7038737 , 0.7069785 , 0.7078975 , 0.7123543 ,
       0.7326614 , 0.74038345, 0.753019  , 0.7606681 , 0.99999994],
      dtype=float32)

In [22]:
import json
with open('../ics_cwe/id_to_pos.json') as fp:
    id_to_pos = json.load(fp)
with open('../ics_cwe/pos_to_id.json') as fp:
    pos_to_id = json.load(fp)

In [25]:
import json
import networkx as nx

# Load the JSON data again
with open('../../graph_network/graph_0.json', 'r') as file:
    a_graph_data = json.load(file)

In [26]:
with open('../../graph_network/graph_Weakness.json') as f:
    w_graph_data = json.load(f)

In [37]:
def get_index_id_mapping(graph, isAttack):
    mapping = {}
    id_tag = 'ID'
    if(isAttack):
        id_tag = 'id'
    for i in range(len(graph['nodes'])):
        mapping[graph['nodes'][i][id_tag]]=i
    return mapping

In [38]:
attack_map = get_index_id_mapping(a_graph_data, True)
weakness_map = get_index_id_mapping(w_graph_data, False)

In [ ]:
# 138-[622-619]

In [59]:
weakness_map[pos_to_id['622']]

419

In [169]:
a_graph_data['nodes'][attack_map[pos_to_id['64']]]

{'modified': '2023-03-09T18:38:51.471Z',
 'name': 'Network Sniffing',
 'description': 'Network sniffing is the practice of using a network interface on a computer system to monitor or capture information (Citation: Enterprise ATT&CK January 2018) regardless of whether it is the specified destination for the information. \n\nAn adversary may attempt to sniff the traffic to gain information about the target. This information can vary in the level of importance. Relatively unimportant information is general communications to and from machines.  Relatively important information would be login information. User credentials may be sent over an unencrypted protocol, such as Telnet, that can be captured and obtained through network packet analysis. \n\nIn addition, ARP and Domain Name Service (DNS) poisoning can be used to capture credentials to websites, proxies, and internal systems by redirecting traffic to an adversary.',
 'kill_chain_phases': [{'kill_chain_name': 'mitre-ics-attack',
   'p

In [170]:
w_graph_data['nodes'][weakness_map[pos_to_id['689']]]

{'ID': '291',
 'Name': 'Reliance on IP Address for Authentication',
 'Abstraction': 'Variant',
 'Structure': 'Simple',
 'Status': 'Incomplete',
 'Description': 'The product uses an IP address for authentication.',
 'Extended_Description': 'IP addresses can be easily spoofed. Attackers can forge the source IP address of the packets they send, but response packets will return to the forged IP address. To see the response packets, the attacker has to sniff the traffic between the victim machine and the forged IP address. In order to accomplish the required sniffing, attackers typically attempt to locate themselves on the same subnet as the victim machine. Attackers may be able to circumvent this requirement by using source routing, but source routing is disabled across much of the Internet today. In summary, IP address verification can be a useful part of an authentication scheme, but it should not be the single factor required for authentication.',
 'Related_Weaknesses': [{'Nature': 'Chi

In [175]:
id_to_pos['923']

446

In [189]:
w_graph_data['nodes'][weakness_map['923']]

{'ID': '923',
 'Name': 'Improper Restriction of Communication Channel to Intended Endpoints',
 'Abstraction': 'Class',
 'Structure': 'Simple',
 'Status': 'Incomplete',
 'Description': 'The product establishes a communication channel to (or from) an endpoint for privileged or protected operations, but it does not properly ensure that it is communicating with the correct endpoint.',
 'Extended_Description': ['Attackers might be able to spoof the intended endpoint from a different system or process, thus gaining the same level of access as the intended endpoint.',
  'While this issue frequently involves authentication between network-based clients and servers, other types of communication channels and endpoints can have this weakness.'],
 'Related_Weaknesses': [{'Nature': 'ChildOf',
   'CWE_ID': '284',
   'View_ID': '1000',
   'Ordinal': 'Primary'}],
 'Mapping_Notes': ['Allowed-with-Review',
  'This CWE entry is a Class and might have Base-level children that would be more appropriate',
 

In [196]:
id_to_pos['290'] #388

688

In [193]:
a = np.argsort(our_emb_matrix[0][446][:203])[::-1]
np.where(a==64)

(array([46]),)

In [197]:
a = np.argsort(text_matrix[0][688][:203])[::-1]
np.where(a==64)

(array([72]),)

In [88]:
1136-517

619

In [89]:
1136-1055

81

In [124]:
def get_top(cosine_matrix, k):
    # Flatten the matrix while keeping track of the indices
    flattened_indices = np.argsort(cosine_matrix, axis=None)[::-1]
    
    # Initialize an empty list to store the valid top 10 pairs
    valid_top_indices = []
    
    # Loop over the sorted indices to filter out diagonal elements
    for flat_index in flattened_indices:
        row, col = np.unravel_index(flat_index, cosine_matrix.shape)
        if (row != col) and ((row<203 and col>203)):
            valid_top_indices.append((row, col))
        if len(valid_top_indices) == k:
            break
    
    # Print the row and column indices of the top 10 maximum values (excluding diagonal)
    print(f"Row and column indices of the top {k} maximum values (excluding diagonal): {valid_top_indices}")
    
    # Optionally, print the corresponding values
    top_values = [cosine_matrix[row, col] for row, col in valid_top_indices]
    print(f"Top {k} maximum values (excluding diagonal): {top_values}")
    return valid_top_indices

In [137]:
top1 = get_top(our_emb_matrix[0],50)

Row and column indices of the top 50 maximum values (excluding diagonal): [(55, 609), (9, 609), (117, 481), (27, 1121), (9, 608), (112, 974), (147, 1105), (156, 707), (55, 608), (64, 1121), (72, 903), (47, 608), (59, 1121), (150, 1105), (64, 689), (45, 953), (41, 609), (108, 1048), (69, 608), (17, 660), (147, 708), (93, 609), (176, 974), (9, 433), (117, 607), (66, 481), (23, 676), (27, 689), (62, 973), (156, 902), (62, 750), (172, 831), (41, 1105), (117, 606), (123, 1105), (24, 402), (62, 692), (80, 410), (34, 410), (109, 609), (109, 1105), (31, 708), (23, 677), (136, 440), (170, 903), (93, 1020), (147, 425), (23, 675), (147, 608), (78, 700)]
Top 50 maximum values (excluding diagonal): [0.90922165, 0.89574367, 0.891054, 0.8893047, 0.8890803, 0.8748248, 0.87027353, 0.86971503, 0.8627675, 0.8622357, 0.8567123, 0.8556904, 0.8545036, 0.85385674, 0.8532069, 0.8510028, 0.85087425, 0.8439738, 0.8435763, 0.8424997, 0.8399693, 0.8393638, 0.8389293, 0.8387298, 0.8375761, 0.8367665, 0.836677, 0.8

In [138]:
top2 = get_top(text_matrix[0],50)

Row and column indices of the top 50 maximum values (excluding diagonal): [(64, 689), (112, 393), (64, 700), (37, 609), (123, 390), (74, 797), (5, 708), (176, 393), (43, 425), (64, 1121), (123, 1020), (109, 390), (64, 797), (74, 689), (85, 425), (177, 538), (137, 700), (45, 390), (69, 390), (5, 390), (71, 708), (47, 609), (74, 700), (10, 1020), (55, 609), (109, 609), (150, 425), (177, 425), (21, 425), (121, 797), (43, 708), (9, 609), (10, 425), (69, 425), (10, 1025), (112, 425), (68, 708), (45, 1120), (10, 390), (45, 209), (5, 425), (105, 391), (64, 725), (55, 215), (69, 609), (45, 1014), (47, 390), (167, 538), (85, 393), (123, 901)]
Top 50 maximum values (excluding diagonal): [0.7224605577264848, 0.6961045823242791, 0.6855438816290164, 0.6817153199707007, 0.6815209218783034, 0.68150253837665, 0.6788426776620691, 0.6786281365870481, 0.6779542877290493, 0.674740289963606, 0.6735925008230782, 0.6689157415709652, 0.666410579994802, 0.6663217070562745, 0.6636631367016123, 0.662195098308637

In [139]:
our_emb_matrix[0][64][689]

0.8532069

In [140]:
set(top1) & set(top2)

{(9, 609), (55, 609), (64, 689), (64, 1121), (109, 609)}